In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/turkish-to-english-translator/TR2EN.txt
/kaggle/input/glove6b/glove.6B.200d.txt
/kaggle/input/glove6b/glove.6B.50d.txt
/kaggle/input/glove6b/glove.6B.300d.txt
/kaggle/input/glove6b/glove.6B.100d.txt


data seti 450k dan oluşan bir dataset

In [80]:
import tensorflow as tf
import numpy as np

In [81]:
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding, CuDNNGRU
from keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [82]:
#model cümle üretmeye başlangıç tokenini gördükten sonra başlıyor. başlangıç tokeni datasette olmayan bir kelime olmalıdır.
#cümlenin sonlanması için bitiş tokenini belirtmemiz gerekiyor.
#boşlum karekterleri çok önemli !!!!

mark_start = 'ssss '
mark_end = ' eeee'

In [83]:
data_src = []
data_dest = []

In [84]:
for line in open('../input/turkish-to-english-translator/TR2EN.txt', encoding='UTF-8'):
    en_text, tr_text = line.rstrip().split('\t')
    tr_text = mark_start + tr_text + mark_end
    data_src.append(en_text)
    data_dest.append(tr_text)

In [85]:
print(data_src[100])
print(data_dest[100])


I drove.
ssss Araba sürdüm. eeee


In [86]:
print(data_src[200000])
print(data_dest[200000])

Can you see anything missing?
ssss Eksik bir şey görebiliyor musun? eeee


In [87]:
class TokenizerWrap(Tokenizer):
    def __init__(self, texts, padding, reverse=False, num_words=None):
        Tokenizer.__init__(self, num_words=num_words) #tokenizer tanımladık
        
        self.fit_on_texts(texts) # aldığımız kelimeleri tokenleştirdik
        
        # tokenleri text haline getirmek için key ve valueların yerini değiştiriyoruz.
        self.index_to_word = dict(zip(self.word_index.values(), self.word_index.keys()))
        
        self.tokens = self.texts_to_sequences(texts) # elimizdeki yazıları tokenler olarak bir listede topluyoruz.
        
        
        #pre padding başa ekleme, post padding sona ekleme, pre truncating baştan çıkarma, post truncating sondan çıkarma
        
        if reverse:
            self.tokens = [list(reversed(x)) for x in self.tokens]
            truncating = 'pre'
        else:
            truncating = 'post'
        
            
        self.num_tokens = [len(x) for x in self.tokens]
        self.max_tokens = np.mean(self.num_tokens) + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)
        
        
        
        
        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)
        
        
    def token_to_word(self, token): # bir sayıya denk gelen kelimeyi döndürüyoruz. 
        word = ' ' if token == 0 else self.index_to_word[token]  #token 0 ise boşluk döndür, değilse dicte key olarak ver valueyu al
        return word
    
    def tokens_to_string(self, tokens): # token listesini cümleye çeviriyoruz. eğer token sıfır değilse çünkü paddingte sıfır eklemiştik
        words = [self.index_to_word[token] for token in tokens if token != 0]
        text = ' '.join(words)
        return text
    
    
    #text_to_tokens funk ile modele tek bir cümle vererek istediğimiz cümleyi modele uygun olarak vereceğiz.
    # modele vereceğimiz cümleyi modele hazır hale getiemk için bir cümle yazalım.
    
    def text_to_tokens(self, text, padding, reverse=False): # modele cümleyi token olarak vermek için hazırlıyoruz.
        tokens = self.texts_to_sequences([text]) # önce aldığımız texti tokenleştiriyoruz.
        tokens = np.array(tokens) # daha sonra bu tokenleri numpy arraye çeviriyoruz.
        
        if reverse:
            tokens = np.flip(tokens, axis=1) #satır üzerinde ters çevirme yapıyoruz.
            truncating = 'pre'
        else:
            truncating = 'post'
            
        tokens = pad_sequences(tokens,
                               maxlen=self.max_tokens,
                               padding=padding,
                               truncating=truncating)
        
        return tokens

In [88]:
tokenizer_src = TokenizerWrap(texts=data_src,
                              padding='pre',
                              reverse=True,
                              num_words=None)



In [89]:
tokenizer_dest = TokenizerWrap(texts=data_dest,
                              padding='post',
                              reverse=False,
                              num_words=None)




# ÖNEMLİ AÇIKLAMA 

> Encodera vereceğimiz inputu tokenleştirme yaparak neden ters çeviriyoruz ?
> 
> bunun iki türlü faydası var:
>     birincisi truncating yaparken cümleyi başından kesiyoruz. eğer cümleyi ters çevirmezsek cümleyi kesince baştaki kelimeler gidecekti. 
>     
>     bunun şöyle bir yararı var:
>         encodera verdiğimiz inputta baştaki kelimelerin kesildiğini düşünelim. encoder çalışıp düşünce vektörü oluştuurp decodera verecek
>         decoderda ise truncatingi post yaptık yani cümlenin sonundaki kelimeler kesiliyor. öyleyse encodera verilecek input bir cümlenin 
>         sonundaki kelimelerden oluşacak, decoderda ise bir cümlenin başındaki kelimelerden oluşacak. yani network ingilizce cümlelerin 
>         sonu   ile türkçe cümlelerin başını eşleştirecek bu şekilde doğru kelimeler birbirleri ile eşleşecek.
>         
>     ikinci avantajı:
>         cümleleri encodera verince en sondan başlayarak başa doğru ilerleyecek encoder düşünce vektörü üretmeden önce gördüğü son şey
>         cümlenin başı olacak. decodera geçtiğinde ise direk cümlenin başından başlayıp üretmeye başlayacak bu şekilde en son cümlenin başını 
>         görüp üretmeye başlarsa daha isabetli bir çeviri yapabilir.
>     

In [90]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape) #ingilizcede cümle boyutumuz 11
print(tokens_dest.shape) #türkçede cümle boyutumuz 10

(473035, 11)
(473035, 10)


In [91]:
print(tokens_dest[200000])

[   1 2391    4   18 4127   48    2    0    0    0]


In [92]:
tokenizer_dest.tokens_to_string(tokens_dest[200000])

'ssss eksik bir şey görebiliyor musun eeee'

In [93]:
print(tokens_src[200000])

[   0    0    0    0    0    0 1028  113   95    5   39]


In [94]:
tokenizer_src.tokens_to_string(tokens_src[200000])

'missing anything see you can'

In [95]:
data_src[200000]

'Can you see anything missing?'

In [96]:
token_start = tokenizer_dest.word_index[mark_start.strip()] #word_indexte kelimelerin tokenleri bulunuyor.
token_start

1

In [97]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end

2

In [98]:
encoder_input_data = tokens_src  # encoder inputu

In [99]:
decoder_input_data = tokens_dest[:, :-1] # baştan başla sondan bir eksiğe kadar git
decoder_output_data = tokens_dest[:, 1:] # 1 den başla sona kadar git.

In [100]:
encoder_input_data[200000]

array([   0,    0,    0,    0,    0,    0, 1028,  113,   95,    5,   39],
      dtype=int32)

In [101]:
decoder_input_data[200000]

array([   1, 2391,    4,   18, 4127,   48,    2,    0,    0], dtype=int32)

In [102]:
decoder_output_data[200000]

array([2391,    4,   18, 4127,   48,    2,    0,    0,    0], dtype=int32)

aşağıda görüldüğü üzere input ve outputta tek fark outputta başlangıç tokeni yok

decoderda inputtan alınan başlangıç tokeniyle eksik kelimesinin üretilmesi beklenir, daha sonra eksik kelimesi input olduğunda 
bir kelimesinin outputta olması gerekir bu olay bitiş tokenine kadar böyle devam eder

In [103]:
tokenizer_dest.tokens_to_string(decoder_input_data[200000])

'ssss eksik bir şey görebiliyor musun eeee'

In [104]:
tokenizer_dest.tokens_to_string(decoder_output_data[200000])

'eksik bir şey görebiliyor musun eeee'

In [105]:
tokenizer_src.tokens_to_string(encoder_input_data[200000])

'missing anything see you can'

In [106]:
num_encoder_words = len(tokenizer_src.word_index)  # eng kelime sayı
num_decoder_words = len(tokenizer_dest.word_index) #tr kelime sayı

In [107]:
print(num_encoder_words)
print(num_decoder_words)


21315
94058


# ENCODER

**Embedding layerda kelime vektörlerini oluşturuken GLOVE kullanacağız.**

elimizde bulunan 21k lık kelimelere karşılık vektör bulamayabiliriz. tüm kelime vektörlerini rastgele tanımlıyoruz.
eğer glove kelime vektörleri içerisinde kelime haznemizdeki kelimelerin vektörleri bulunuyorsa o vektör rastgele oluşturulan vektörün yerine geçicek.

400.000 kelimenin bulunduğu glove vektörünün içinde bizim kelimelerimiz bulunacaktır.

In [108]:
embedding_size = 100   # glove vektörlerinin uzunluğuda 100 

**önce boş bir sözlük oluşturuyoruz, -->> word2vec={}**

**daha sonra kelime ve vektörü olacak şekilde içini dolduruyoruz.**

In [109]:
word2vec = {}
with open('../input/glove6b/glove.6B.100d.txt', encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

In [110]:
#burada yaptığım işlem embedding vektörün içini dolduruyorum eğer kelime glove vektörünün içinde varsa o vektörü alıyorum

embedding_matrix = np.random.uniform(-1, 1, (num_encoder_words, embedding_size))
for word, i in tokenizer_src.word_index.items():
    if i < num_encoder_words:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [111]:
embedding_matrix.shape

(21315, 100)

In [112]:
encoder_input = Input(shape=(None,), name='encoder_input') # ınput layera int tokenleri veriyoruz. #shape none vererek boyutla ilgili sorunları çözüyoruz

In [113]:
#kerasın embedding layerına ekliyoruz.
encoder_embedding = Embedding(input_dim=num_encoder_words, 
                              output_dim=embedding_size,
                              weights=[embedding_matrix],
                              trainable=True,
                              name='encoder_embedding')

In [114]:
state_size = 256 # butün GRU layerlar 256 boyutlu output üretecek. daha yüksek bir state_size ile daha iyi sonuç alınır.

In [115]:
#3 katmanlı bir GRU yapısı oluşturuyoruz.
encoder_gru1 = GRU(state_size, name='encoder_gru1', return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2', return_sequences=True)
encoder_gru3 = GRU(state_size, name='encoder_gru3', return_sequences=False)  #düşünce vektörünü oluşturuyor.

In [116]:
#layerları connect_encoder()' da zincir gibi bağlıyoruz.
def connect_encoder():
    net = encoder_input
    
    net = encoder_embedding(net)
    
    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)
    
    encoder_output = net
    
    return encoder_output


# encoder_output ile düşünce vektörünü oluşturmuş olduk . artık bu fonk çağırarak düşünce vektörüne ulaşabiliriz.

In [117]:
encoder_output = connect_encoder()

# DECODER

Encodera ingilizce cümleleri Decodera bu ingilizce cümlelerin türkçe çevirilerini veriyoruz.

eğitimde decodera düşünce vektörünü ve ingilizce cümleye karşılık gelen türkçe cümleyi veriyoruz ve modeli eğitiyoruz.

eğitim tamamlandıktan sonra encodera verdiğimiz ing cümlenin sonuda aldığımız düşünce vektörünü başlanğıç tokeni ile birlikte decodera vereceğiz.

In [118]:
decoder_initial_state = Input(shape=(state_size,), name='decoder_initial_state')
    
# decodera vereceğimiz düşünce vektörü için bir input layer oluştuuryoruz.

In [119]:
decoder_input = Input(shape=(None,), name='decoder_input') 

# bu input ise decodera vereceğimiz türkçe cümle için, shape ' e none veriyoruz gelen her input kabulümüz.

In [120]:
#normalde glove ile embedding matrisi oluşturuyoduk ama elimizdeki glove datası ing bizim decoderımızda türkçe veriler var
#ya yeni bir glove modeli eğiteceğiz yada yaptığım gibi rastgele dolduracağız.

decoder_embedding = Embedding(input_dim=num_decoder_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

In [121]:
decoder_gru1 = GRU(state_size, name='decoder_gru1', return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2', return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3', return_sequences=True)

# decoderda üçüncü layera true dedik çünkü buradan bize bir cümle dönüyor, encoderda bize bir vektör dönüyordu.

In [122]:
# decoderın çalışması sonucunda kelimeler elde etmek istiyoruz. suan layerın çalışması durumunda 256 boyutunda vektörler elde edeceğiz.
# bu vektörleri kelimeye çevirmemiz lazım bunun için dense layer oluşturuyoruz. GRUdan dönen sonucu one hot türünde arraye dönüştüreceğiz.
# bu vektörde en büyük değere sahip elemanın indexi output olarak hangi kelimenin verilmesi gerektiğini gösteriyor.
#aktivasyonu linear yapınca değerler olduğu gibi geçiyor.


decoder_dense = Dense(num_decoder_words,
                      activation='linear',
                      name='decoder_output')

In [123]:
#initial_state encoderın ürettiği düşünce vektörünü veriyoruz.

def connect_decoder(initial_state):
    net = decoder_input
    
    net = decoder_embedding(net)
    
    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)
    
    decoder_output = decoder_dense(net)
    
    return decoder_output

In [124]:
decoder_output = connect_decoder(initial_state=encoder_output)  # encoder_outputu (düşünce vektörünü ) -> decodera veriyoruz.

model_train = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output]) 

#decodera iki farklı input veriyoruz bunlardan ilki düşünce vektörü diğeri türkçe cümle.
#model_train ile encoder ve decoder birbirleri ile bağlanmış oldu.

In [125]:
model_encoder = Model(inputs=[encoder_input], outputs=[encoder_output])    # çeviri için yapıyoruz.

In [126]:
# iki farklı model daha tanımlayacağız bunları ise model tamamlandıktan sonra çeviri için kullancağız. 

decoder_output = connect_decoder(initial_state=decoder_initial_state)
model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs=[decoder_output])

In [127]:
#üretilen türkçe kelimeyi gerçekteki türkçe kelime ile karşılaştırmalıyız 
#şimdi her kelime için one hot vektör tanımlayıp bunun karşılaştırılması yapılabilir ama inanılmaz büyük bir boyuta sahip olucak
# bu yüzden de bunu kullanmak yerine spars cross entropy kullanıyoruz.

# sparse cross entropy --> her iterasyonda kendi içerisinde intleri one hot vektörleri çeviriyor bu sayede kocaman bir matris oluşturmuyoruz.
# spars entropyde softmaxten geçiyor bu yüzden decoderda dense layerda softmax kullanmadık. lineer kullandık


def sparse_cross_entropy(y_true, y_pred):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    loss_mean = tf.reduce_mean(loss)
    return loss_mean


In [128]:
optimizer = RMSprop(lr=1e-3)

In [129]:
import tensorflow.compat.v1 as tf

#import tensorflow as tf
tf.compat.v1.disable_eager_execution()

#tf.disable_v2_behavior()
#x = tf.placeholder(shape=[None, None], dtype=tf.int32)

In [130]:
#decoder_target = tf.placeholder(dtype=tf.int32, shape=(None,None))
decoder_target=tf.compat.v1.placeholder(shape=[None, None], dtype=tf.int32)

In [131]:
#model_trainde encoder ve decoderı bağlamıştık zaten

model_train.compile(optimizer=optimizer,
                    loss=sparse_cross_entropy,
                    target_tensors=[decoder_target])

In [132]:
#modelin eğitimi uzun süreceğinden kayıt ediyoruz.

path_checkpoint = 'checkpoint.keras'
checkpoint = ModelCheckpoint(filepath=path_checkpoint, save_weights_only=True)

In [133]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print('Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.')
    print(error)

Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.
Unable to open file (unable to open file: name = 'checkpoint.keras', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


In [134]:
# modele vereceğimiz inputları belirtiyoruz. bu inputlar int tokenlerden oluşuyor.

x_data = {'encoder_input': encoder_input_data, 'decoder_input': decoder_input_data}

In [135]:
#modelden sonuç beklediğimiz outputu belirtiyoruz.

y_data = {'decoder_output': decoder_output_data}

In [136]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=256,
                epochs=10,
                callbacks=[checkpoint])

Train on 473035 samples
Epoch 1/10
473035/473035 [==============================] - 269s 568us/sample - loss: 3.2297
Epoch 2/10
473035/473035 [==============================] - 265s 561us/sample - loss: 2.2303
Epoch 3/10
473035/473035 [==============================] - 267s 564us/sample - loss: 1.9414
Epoch 4/10
473035/473035 [==============================] - 264s 559us/sample - loss: 1.8087
Epoch 5/10
473035/473035 [==============================] - 263s 555us/sample - loss: 1.7816
Epoch 6/10
473035/473035 [==============================] - 263s 555us/sample - loss: 1.7550
Epoch 7/10
473035/473035 [==============================] - 263s 556us/sample - loss: 1.7234
Epoch 8/10
473035/473035 [==============================] - 263s 556us/sample - loss: 1.6937
Epoch 9/10
473035/473035 [==============================] - 265s 560us/sample - loss: 1.6684
Epoch 10/10
473035/473035 [==============================] - 265s 561us/sample - loss: 1.6462


In [137]:
# 3 farklı model oluşturduk ama bunların weightleri aynı. model trainin encoderı ile oluşturduğumuz model encoderın weightleri aynı
#eğitim aşamasında decodera farklı input ve output verdiğimiz için aynı modeli farklı şekillerde tanımlıyoruz.

#model traine ingilizce cümleleri ve bu cümlelerin türkçe çevirileri input olarak vermiştik. modelimizi test ederken verdiğimiz ingilizce 
#cümlenin türkçe çevirisini verirsek sonucun pek anlamıyor olmuyor. model train bizden hem encoder hemde decoder için bizden input istiyor

# ing cümleyi model encodera veriyoruz. bize bir düşünce vektörü verecek elde ettiğimiz düşünce vektörünü ve başlangıç tokenini 
#model decodera veriyoruz.

In [138]:
def translate(input_text, true_output_text=None):
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                reverse=True,
                                                padding='pre')
    
    initial_state = model_encoder.predict(input_tokens)       # düşünce vektörünü ürettik
    
    max_tokens = tokenizer_dest.max_tokens 
    
    decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=np.int)  # normalde bachsize 256 ama suan 1 cümle üzerinden çalışıyoruz.
    
    token_int = token_start
    output_text = ''
    count_tokens = 0
    
    while token_int != token_end and count_tokens < max_tokens:
        decoder_input_data[0, count_tokens] = token_int
        x_data = {'decoder_initial_state': initial_state, 'decoder_input': decoder_input_data}
        
        decoder_output = model_decoder.predict(x_data)
        
        token_onehot = decoder_output[0, count_tokens, :]
        token_int = np.argmax(token_onehot)
        
        sampled_word = tokenizer_dest.token_to_word(token_int)
        output_text += ' ' + sampled_word
        count_tokens += 1
        
    print('Input text:')
    print(input_text)
    print()
    
    print('Translated text:')
    print(output_text)
    print()
    
    if true_output_text is not None:
        print('True output text:')
        print(true_output_text)
        print()

In [139]:
translate(input_text=data_src[400000], true_output_text=data_dest[400000])

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Input text:
You are telling it second hand, aren't you?

Translated text:
 onu ikinci şekilde söylüyorsun değil mi eeee

True output text:
ssss Onu dolaylı olarak anlatıyorsun, değil mi? eeee



In [148]:
translate(input_text=data_src[300000], true_output_text=data_dest[300000])

Input text:
Tom will certainly try to do that.

Translated text:
 tom kesinlikle bunu yapmaya çalışacak eeee

True output text:
ssss Tom kesinlikle onu yapmaya çalışacak. eeee



In [149]:
translate(input_text=data_src[2000], true_output_text=data_dest[2000])

Input text:
I feel weak.

Translated text:
 ben zayıf hissediyorum eeee

True output text:
ssss Cılız hissediyorum. eeee



In [140]:
translate(input_text='Why people are so rude')

Input text:
Why people are so rude

Translated text:
 i̇nsanlar neden bu kadar kaba eeee



In [147]:
translate(input_text='This summer ı went to Turkey')

Input text:
This summer ı went to cinema

Translated text:
 bu yaz yaz tom'un çıktı eeee



In [144]:
translate(input_text='Where did you buy this dress?')

Input text:
Where did you buy this dress?

Translated text:
 bu elbiseyi nereden aldın eeee



In [145]:
translate(input_text='which road leads to airport')

Input text:
which road leads to airport

Translated text:
 hangi yol havaalanına gider eeee

